# Import libraries

In [1]:
from helpers import SEABORN_COLORS, create_plot
import os
import sys
import time
import datetime

from termcolor import colored, cprint

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import sys
from os import listdir

import numpy as np
import pandas as pd  # Import Pandas and Numpy to create databases
import tensorflow as tf  # Import tensorflow to do tensor manipulations
from tensorflow import keras
from tensorflow.keras import Input, Model, layers, regularizers
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import csv
import statsmodels.api as sm
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Paths

In [2]:
PATH_data = f"/mnt/d/UKB_toydata/"
PATH_GENO = f"{PATH_data}genotype"
PATH_usefull = f"{PATH_data}usefull_values_2"
PATH_usefull_accuracy = f"{PATH_usefull}/accuracy_looping2/"
os.system(f"mkdir {PATH_usefull_accuracy}")
PATH_PHENO = f"{PATH_data}phenotype"

mkdir: cannot create directory ‘/mnt/d/UKB_toydata/usefull_values_2/accuracy_looping2/’: File exists


In [3]:
phenotypes = ['bc', 'cad', 'ibd', 'ms', 'pc', 'ra', 't2d']

In [4]:
#bottles = list(range(100))[1:]
bottles = [2, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
epoch = 60

In [5]:
for disease in phenotypes:
    path = f"{PATH_usefull}/no_corr/{disease}"
    sumstat = pd.read_pickle(f"{path}/sumstat.pkl")
    pheno = pd.read_pickle(f"{path}/pheno_PRS.pkl")
    nr_significant_snps = 300
    SNPS = sumstat.sort_values(by=['-logp'])[['snp','#CHROM']].tail(nr_significant_snps)
    SNPS = SNPS.sort_values(by=['#CHROM'])
    grouped = SNPS.groupby('#CHROM')

    # Iterate over groups and create DataFrames
    dataframes = {}
    for name, group in grouped:
        dataframes[name] = group.copy()

    name_file = "total_pop"
    # Access each created DataFrame using the unique value as the key
    genos_dfs = []
    for chrom, dataframe in dataframes.items():
        print(f"DataFrame for chrom {chrom}:\n{dataframe}\n")
        # Define a function to extract ID from snp
        def extract_id(s):
            return s[:-2]  # Remove the last two characters

        # Apply the function to create the 'ID' column
        dataframe['ID'] = dataframe['snp'].apply(extract_id)
        snp_ids_to_keep = " ,".join(list(dataframe['ID']))
        path_total_pop = PATH_GENO + f"/02_per_chrom/total_pop_chr_{chrom}"
        os.system(f"plink2 --bfile {path_total_pop} --snps {snp_ids_to_keep} --recode A --out ./temp2")
        genos = pd.read_csv('./temp2.raw', sep="\t")
        genos = genos.fillna(0)
        to_drop = ['IID','PAT','MAT','SEX', 'PHENOTYPE']
        genos = genos.drop(to_drop, axis=1).astype(int) # encode it as (-1, 0, 1)
        genos_dfs.append(genos)
    merged_df = genos_dfs[0]
    for df in genos_dfs[1:]:
        merged_df = pd.merge(merged_df, df, on='FID')
    
    geno = pd.merge(pheno[['FID','phenotype','PRS']], merged_df, on='FID')
    
    # Split DataFrame into controls to train the model and cases and controls to calculate PRS
    cases_df = geno[geno['phenotype'] == 1]
    cases_df = cases_df.sample(frac=1, random_state=42)
    train_cases = cases_df.head(len(cases_df)//2)
    replicated_cases = cases_df.tail(len(cases_df) - len(cases_df)//2)

    controls_df = geno[geno['phenotype'] == 0]
    controls_df = controls_df.sample(frac=1, random_state=42)
    train_controls = controls_df.head(len(controls_df)//2)
    replicated_controls = controls_df.tail(len(controls_df) - len(controls_df)//2)

    smallest_size = min(len(train_cases), len(replicated_cases) ,len(train_controls), len(replicated_controls))

    train_cases = train_cases.head(smallest_size)
    replicated_cases = replicated_cases.head(smallest_size)
    train_controls = train_controls.head(smallest_size)
    replicated_controls = replicated_controls.head(smallest_size)
    
    geno = pd.concat([train_cases,train_controls], axis = 0)
    to_drop = ['FID','phenotype','PRS']
    geno_input = geno.drop(to_drop, axis=1).astype(int) -1
    pheno_input = geno[['phenotype']]
    
    input_shape_geno = geno_input.shape
    input_layer_geno = Input(shape=input_shape_geno[1:], name='input_geno')

    input_shape_pheno = pheno_input.shape
    input_layer_pheno = Input(shape=input_shape_pheno[1:], name='input_pheno')
    for bottle in bottles:
        size_layer_1 = int(round(input_shape_geno[1] + bottle)/2)

        # Define your regularization strength (lambda)
        l2_lambda = 0.001  # Adjust this value as needed

        # Create layers

        # Create layers
        encoder_init_1 = layers.Dense(size_layer_1, 
                               activation="elu", 
                               name="encoder_init_1",
                               kernel_regularizer=regularizers.l2(l2_lambda))

        encoder_init_2 = layers.Dense(bottle, 
                               activation="linear", 
                               name="encoder_init_2",
                               kernel_regularizer=regularizers.l2(l2_lambda))

        decoder_init_1 = layers.Dense(size_layer_1, 
                               activation="elu", 
                               name="decoder_init_1",
                               kernel_regularizer=regularizers.l2(l2_lambda))

        decoder_init_2 = layers.Dense(input_shape_geno[1], 
                               activation="linear", 
                               name="decoder_init_2",
                               kernel_regularizer=regularizers.l2(l2_lambda))



        encoder_concat_1 = layers.Dense(size_layer_1, 
                               activation="elu", 
                               name="encoder_concat_1",
                               kernel_regularizer=regularizers.l2(l2_lambda))

        encoder_concat_2 = layers.Dense(input_shape_pheno[1], 
                               activation="sigmoid", 
                               name="encoder_concat_2",
                               kernel_regularizer=regularizers.l2(l2_lambda))

        decoder_concat_1 = layers.Dense(size_layer_1, 
                               activation="linear", 
                               name="decoder_concat_1",
                               kernel_regularizer=regularizers.l2(l2_lambda))

        decoder_concat_2 = layers.Dense(input_shape_geno[1] + bottle, 
                               activation="linear", 
                               name="decoder_concat_2",
                               kernel_regularizer=regularizers.l2(l2_lambda))
        
        bottle_neck = encoder_init_2(encoder_init_1(input_layer_geno))
        concatenated_bottle = tf.concat([bottle_neck, input_layer_pheno], axis=1)
        concatenated_inputs = tf.concat([bottle_neck, input_layer_geno], axis=1)
        
        shape_zeros = concatenated_inputs.shape
        dec_init_1 = decoder_init_1(concatenated_bottle)
        X2 = decoder_init_2(dec_init_1)
        y = encoder_concat_2(encoder_concat_1(concatenated_inputs))
        conc_X2 = decoder_concat_2(decoder_concat_1(y))
        squared_difference = tf.square(conc_X2 - concatenated_inputs) 
        model = Model(inputs = [input_layer_geno, input_layer_pheno], outputs = [X2, y, squared_difference], name = "fishy")
    
        genos = geno.sample(frac=1)
        pheno = genos[['phenotype']].astype(int)
        genos = genos.drop(to_drop, axis=1)
        len_test = round(genos.shape[0]/10)*3
        len_train = genos.shape[0] - len_test
        X_train = genos.head(len_train) - 1
        X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
        X_test = genos.tail(len_test) - 1
        X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)

        shape_train = (len_train,squared_difference.shape[1])
        shape_test = (len_test,squared_difference.shape[1])
        
        pheno_tensor_train  = tf.convert_to_tensor(pheno.head(len_train))
        pheno_tensor_test = tf.convert_to_tensor(pheno.tail(len_test))
        
        optimizer='Adam'
        loss = 'mse'
        zero_tensor_train = tf.zeros(shape_train, dtype=tf.float32)
        zero_tensor_test = tf.zeros(shape_test, dtype=tf.float32)
        model.compile(optimizer,  loss=['mse', 'binary_crossentropy', 'mse'], metrics = ["mse"])
        
        history = model.fit([X_train,pheno_tensor_train],
                    [X_train, pheno_tensor_train, zero_tensor_train],
                    validation_data=([X_test, pheno_tensor_test], 
                                     [X_test, pheno_tensor_test, zero_tensor_test]),
                    epochs = epoch,
                    verbose = 0)
        genos = geno.sample(frac=1)
        pheno = genos[['phenotype']].astype(int)
        genos = genos.drop(to_drop, axis=1)
        
        genos = genos - 1
        
        X1 = tf.convert_to_tensor(genos, dtype=tf.float32)
        X2 = tf.convert_to_tensor(pheno, dtype=tf.float32)
        output_model = model([X1,X2])
        submodel_layers1 = model.layers[:3]
        submodel1 = Model(inputs=[model.input[0]], outputs=submodel_layers1[-1].output)
        submodel1(X1)
        
        submodel_layers2 = model.layers[:6]
        submodel2 = Model(inputs=[model.input[0]], outputs=submodel_layers2[-1].output)
        submodel2(X1)
        
        submodel_layers3 = model.layers[:16]
        submodel3 = Model(inputs=[model.input[0]], outputs=submodel_layers3[-1].output)
        submodel3(X1)
        
        cols = []
        for i2 in range(np.shape(submodel1(X1))[1]):
            cols.append(f'dim{i2+1}')
        bottleneck_1 = pd.DataFrame(submodel1(X1), columns = cols)
        cols = []
        for i2 in range(np.shape(submodel2(X1))[1]):
            cols.append(f'pheno{i2+1}')
        predicted_pheno = pd.DataFrame(submodel2(X1), columns = cols)
        
        bottleneck_1['phenotype'] = list(pheno['phenotype'])
        bottleneck_1['predicted phenotype'] = list(predicted_pheno['pheno1'])
        
        # Split the data into features (X) and labels (Y)
        X = bottleneck_1[['predicted phenotype']]
        Y = bottleneck_1['phenotype']

        # Split the data into training and testing sets
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
        # Instantiate the model
        model_log = LogisticRegression()

        # Fit the model to the training data
        model_log.fit(X_train, Y_train)
        y_pred = model_log.predict(X_test)

        accuracy_sc = accuracy_score(Y_test, y_pred)

        test_accuracy = np.round(accuracy_sc,3)
        bottleneck_1.to_pickle(f"{PATH_usefull_accuracy}/{disease}_dims_{bottle}_test_accuracy_{test_accuracy}.pkl")
        
        
        # NOW VALIDATION SET
        geno = pd.concat([replicated_cases,train_controls], axis = 0)
        genos = geno.sample(frac=1)
        pheno = genos[['phenotype']].astype(int)
        genos = genos.drop(to_drop, axis=1)
        genos = genos - 1
        
        X1 = tf.convert_to_tensor(genos, dtype=tf.float32)
        X2 = tf.convert_to_tensor(pheno, dtype=tf.float32)
        output_model = model([X1,X2])
        submodel_layers1 = model.layers[:3]
        submodel1 = Model(inputs=[model.input[0]], outputs=submodel_layers1[-1].output)
        submodel_layers2 = model.layers[:6]
        submodel2 = Model(inputs=[model.input[0]], outputs=submodel_layers2[-1].output)
        
        cols = []
        for i2 in range(np.shape(submodel1(X1))[1]):
            cols.append(f'dim{i2+1}')
        bottleneck_1 = pd.DataFrame(submodel1(X1), columns = cols)
        cols = []
        for i2 in range(np.shape(submodel2(X1))[1]):
            cols.append(f'pheno{i2+1}')
        predicted_pheno = pd.DataFrame(submodel2(X1), columns = cols)
        
        bottleneck_1['phenotype'] = list(pheno['phenotype'])
        bottleneck_1['predicted phenotype'] = list(predicted_pheno['pheno1'])
        
        # Split the data into features (X) and labels (Y)
        X = bottleneck_1[['predicted phenotype']]
        Y = bottleneck_1['phenotype']

        # Split the data into training and testing sets
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
        # Instantiate the model
        #model_log = LogisticRegression()

        # Fit the model to the training data
        #model_log.fit(X_train, Y_train)
        y_pred = model_log.predict(X_test)

        accuracy_sc = accuracy_score(Y_test, y_pred)

        test_accuracy = np.round(accuracy_sc,3)
        bottleneck_1.to_pickle(f"{PATH_usefull_accuracy}/{disease}_dims_{bottle}_validation_accuracy_{test_accuracy}.pkl")
        
    # PRS accuracy
    pheno = pd.read_pickle(f"{path}/pheno_PRS.pkl")
    # Split DataFrame into controls to train the model and cases and controls to calculate PRS
    cases_df = pheno[pheno['phenotype'] == 1]
    controls_df = pheno[pheno['phenotype'] == 0]
    controls_df = controls_df.sample(frac=1, random_state=42)
    train_controls = controls_df.head(len(controls_df)//2)
    replicated_controls = controls_df.tail(len(controls_df) - len(controls_df)//2)
    smallest_size = min(len(train_controls), len(replicated_controls), len(cases_df))


    cases_df = cases_df.head(smallest_size)
    train_controls = train_controls.head(smallest_size)
    replicated_controls = replicated_controls.head(smallest_size)
    
    pheno = pd.concat([cases_df, replicated_controls], axis = 0)
    # Split the data into features (X) and labels (Y)
    X = pheno[['PRS']]
    Y = pheno['phenotype']
    
    # Split the data into training and testing sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    # Instantiate the model
    model_log = LogisticRegression()

    # Fit the model to the training data
    model_log.fit(X_train, Y_train)
    y_pred = model_log.predict(X_test)

    accuracy_sc = accuracy_score(Y_test, y_pred)
            
    test_accuracy = np.round(accuracy_sc,3)
    pheno.to_pickle(f"{PATH_usefull_accuracy}/{disease}_dims_{bottle}_classical_PRS_accuracy_{test_accuracy}.pkl")
        

DataFrame for chrom 21:
                     snp #CHROM
7481   21:22313265:C_T_C     21
11139  21:23975186:A_C_A     21
8576   21:33651490:A_G_A     21
2253   21:36191365:A_G_G     21
11029  21:36080290:C_T_C     21
...                  ...    ...
8805   21:35979307:C_T_C     21
2681   21:33583062:C_G_G     21
12688  21:16263419:C_G_C     21
10395  21:36064321:C_T_T     21
1387   21:35554684:A_G_G     21

[109 rows x 2 columns]

PLINK v2.00a5LM 64-bit Intel (21 Jun 2023)     www.cog-genomics.org/plink/2.0/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ./temp2.log.
Options in effect:
  --bfile /mnt/d/UKB_toydata/genotype/02_per_chrom/total_pop_chr_21
  --export A
  --out ./temp2
  --snps 21:22313265:C_T ,21:23975186:A_C ,21:33651490:A_G ,21:36191365:A_G ,21:36080290:C_T ,21:47800628:A_G ,21:30986448:C_T ,21:45365248:C_G ,21:16353531:C_T ,21:36074558:A_C ,21:16288177:G_T ,21:34263801:A_G ,21:43865313:C_T ,21:35826877:C_T ,21:29362621:C_T ,21:36

--export A pass 1/1: loading... 619writing... 10101111121213131414151516161717181819202021212222232324242525262627272828292930303131323233333434353536363737383839404041414242434344444545464647474848494950505151525253535454555556565757585859606061616262636364646565666667676868696970707171727273737474757576767777787879808081818282838384848585868687878888898990909191929293939494959596969797989899done.
--export A: ./temp2.raw written.
End time: Tue Sep 19 16:44:23 2023
DataFrame for chrom 21:
                     snp #CHROM
730    21:14916055:A_T_T     21
6291   21:41247892:A_G_A     21
2751   21:41512399:C_T_T     21
6432   21:16664777:A_G_G     21
12957  21:22012401:C_T_T     21
...                  ...    ...
6282   21:45119629:C_T_C     21
11201  21:44863777:A_C_A     21
7530   21:19190427:C_T_T     21
7280   21:24714337:A_G_G     21
8935   21:16791374:C_T_C     21

[183 rows x 2 columns]

PLINK v2.00a5LM 64-bit Intel (21 Jun 2023)     www.cog-genomics.org/plink/2.0/
(C) 2005-2023 Shau

--export A pass 1/1: loading... 549writing... 10101111121213131414151516161717181819202021212222232324242525262627272828292930303131323233333434353536363737383839404041414242434344444545464647474848494950505151525253535454555556565757585859606061616262636364646565666667676868696970707171727273737474757576767777787879808081818282838384848585868687878888898990909191929293939494959596969797989899done.
--export A: ./temp2.raw written.
End time: Tue Sep 19 16:55:45 2023
DataFrame for chrom 21:
                     snp #CHROM
9490   21:18285438:C_G_C     21
6255   21:44721579:C_T_C     21
7790   21:33410631:A_G_A     21
6076   21:21030200:C_T_T     21
6074   21:21030508:C_T_T     21
...                  ...    ...
5228   21:39656505:C_T_T     21
199    21:32304057:A_G_G     21
7900   21:32848102:G_T_G     21
10284  21:21078149:A_G_A     21
12681  21:21171177:C_T_C     21

[156 rows x 2 columns]

PLINK v2.00a5LM 64-bit Intel (21 Jun 2023)     www.cog-genomics.org/plink/2.0/
(C) 2005-2023 Shau

--export A pass 1/1: loading... 479writing... 10101111121213131414151516161717181819202021212222232324242525262627272828292930303131323233333434353536363737383839404041414242434344444545464647474848494950505151525253535454555556565757585859606061616262636364646565666667676868696970707171727273737474757576767777787879808081818282838384848585868687878888898990909191929293939494959596969797989899done.
--export A: ./temp2.raw written.
End time: Tue Sep 19 16:59:26 2023
DataFrame for chrom 21:
                     snp #CHROM
3230   21:40283267:C_T_T     21
11422  21:37913465:A_G_A     21
7      21:47986083:A_G_G     21
8752   21:24827179:G_T_T     21
3470   21:43182633:C_T_C     21
...                  ...    ...
2588   21:46270245:A_G_G     21
5356   21:35366219:A_G_A     21
3573   21:27255117:A_C_A     21
5000   21:20919209:A_G_G     21
5536   21:35419190:C_T_T     21

[123 rows x 2 columns]

PLINK v2.00a5LM 64-bit Intel (21 Jun 2023)     www.cog-genomics.org/plink/2.0/
(C) 2005-2023 Shau

--export A pass 1/1: loading... 309writing... 10101111121213131414151516161717181819202021212222232324242525262627272828292930303131323233333434353536363737383839404041414242434344444545464647474848494950505151525253535454555556565757585859606061616262636364646565666667676868696970707171727273737474757576767777787879808081818282838384848585868687878888898990909191929293939494959596969797989899done.
--export A: ./temp2.raw written.
End time: Tue Sep 19 17:01:16 2023
DataFrame for chrom 21:
                     snp #CHROM
7461   21:31385507:A_G_A     21
6700   21:39015290:C_G_C     21
2042   21:23555614:A_G_G     21
12173  21:15765902:A_C_C     21
2238   21:29803501:C_T_T     21
...                  ...    ...
12191  21:15720789:C_T_C     21
8479   21:27973721:A_G_G     21
5964   21:37321331:A_T_T     21
6099   21:43092313:C_T_T     21
11103  21:44182554:C_T_C     21

[130 rows x 2 columns]

PLINK v2.00a5LM 64-bit Intel (21 Jun 2023)     www.cog-genomics.org/plink/2.0/
(C) 2005-2023 Shau

--export A pass 1/1: loading... 599writing... 10101111121213131414151516161717181819202021212222232324242525262627272828292930303131323233333434353536363737383839404041414242434344444545464647474848494950505151525253535454555556565757585859606061616262636364646565666667676868696970707171727273737474757576767777787879808081818282838384848585868687878888898990909191929293939494959596969797989899done.
--export A: ./temp2.raw written.
End time: Tue Sep 19 17:05:32 2023
DataFrame for chrom 21:
                     snp #CHROM
9045   21:43080711:G_T_G     21
6150   21:33381040:A_G_G     21
11843  21:25680307:A_C_C     21
388    21:46511411:C_G_C     21
5418   21:42265074:C_T_T     21
...                  ...    ...
12803  21:34264385:A_T_T     21
56     21:30986448:C_T_C     21
6266   21:42428264:A_C_C     21
7082   21:46160661:C_T_C     21
11814  21:39537344:C_G_C     21

[116 rows x 2 columns]

PLINK v2.00a5LM 64-bit Intel (21 Jun 2023)     www.cog-genomics.org/plink/2.0/
(C) 2005-2023 Shau

--export A pass 1/1: loading... 0%46%96% writing... 0%0%1%1%2%2%3%3%4%4%5%5%6%6%7%7%8%8%9%9%10%10%11%11%12%12%13%13%14%14%15%15%16%16%17%17%18%18%19%20%20%21%21%22%22%23%23%24%24%25%25%26%26%27%27%28%28%29%29%30%30%31%31%32%32%33%33%34%34%35%35%36%36%37%37%38%38%39%40%40%41%41%42%42%43%43%44%44%45%45%46%46%47%47%48%48%49%49%50%50%51%51%52%52%53%53%54%54%55%55%56%56%57%57%58%58%59%60%60%61%61%62%62%63%63%64%64%65%65%66%66%67%67%68%68%69%69%70%70%71%71%72%72%73%73%74%74%75%75%76%76%77%77%78%78%79%80%80%81%81%82%82